<!--<badge>--><a href="https://colab.research.google.com/github/gradsflow/gradsflow/blob/main/examples/nbs/06-AutoModel_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

In [ ]:
import os

# you can remove this
os.chdir("../../")

In [ ]:
from gradsflow import Model
from gradsflow.tuner.tuner import Tuner

In [ ]:
from timm import create_model
from gradsflow.data.image import get_fake_data

from gradsflow import AutoDataset


In [ ]:
image_size = (128, 128)
fake_data = get_fake_data(image_size, num_workers=0)
train_ds, train_dl = fake_data['ds'], fake_data['dl']

fake_data = get_fake_data(image_size, num_workers=0)
val_ds, val_dl = fake_data['ds'], fake_data['dl']

num_classes = train_ds.num_classes
autodataset = AutoDataset(train_dl, val_dl, num_classes=num_classes)

In [ ]:
from gradsflow.tuner.tuner import Tuner
from gradsflow.tuner.automodel import AutoModelV2

In [ ]:
cnn1 = create_model("resnet18", pretrained=False, num_classes=num_classes)
cnn2 = create_model("efficientnet_b0", pretrained=False, num_classes=num_classes)

In [ ]:
tuner = Tuner()

cnns = tuner.suggest_complex("learner", cnn1, cnn2)
optimizers = tuner.choice("optimizer", "adam", "sgd")
loss = tuner.choice(
    "loss",
    "crossentropyloss",
)

model = AutoModelV2(cnns)

In [ ]:
model.hp_tune(tuner, autodataset, epochs=4)

2021-09-29 19:43:53,414	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
2021-09-29 19:43:56,265	WARNING function_runner.py:558 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-09-29 19:43:58,540	WARNING worker.py:1215 -- Warning: The actor ImplicitFunc is very large (58 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


Trial name,status,loc,learner,loss,optimizer
_hp_optimizer_7c10a_00000,RUNNING,,1,crossentropyloss,sgd


Result for _hp_optimizer_7c10a_00000:
  date: 2021-09-29_19-44-38
  done: false
  experiment_id: 1f6a6823b4544df88fecde2ffef3b208
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 1
  node_ip: 192.168.120.84
  pid: 2207
  should_checkpoint: true
  time_since_restore: 33.17194175720215
  time_this_iter_s: 33.17194175720215
  time_total_s: 33.17194175720215
  timestamp: 1632924878
  timesteps_since_restore: 0
  train_loss: 2.4102004801986
  training_iteration: 1
  trial_id: 7c10a_00000
  val_accuracy: 0.11
  val_loss: 285.92313784330526
  


Trial name,status,loc,learner,loss,optimizer,iter,total time (s),val_loss,val_accuracy,train_loss
_hp_optimizer_7c10a_00000,RUNNING,192.168.120.84:2207,1,crossentropyloss,sgd,1,33.1719,285.923,0.11,2.4102


Result for _hp_optimizer_7c10a_00000:
  date: 2021-09-29_19-45-11
  done: false
  experiment_id: 1f6a6823b4544df88fecde2ffef3b208
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 2
  node_ip: 192.168.120.84
  pid: 2207
  should_checkpoint: true
  time_since_restore: 66.14079689979553
  time_this_iter_s: 32.968855142593384
  time_total_s: 66.14079689979553
  timestamp: 1632924911
  timesteps_since_restore: 0
  train_loss: 2.2884897565612823
  training_iteration: 2
  trial_id: 7c10a_00000
  val_accuracy: 0.11
  val_loss: 104.01406013601054
  


...

Trial name,status,loc,learner,loss,optimizer,iter,total time (s),val_loss,val_accuracy,train_loss
_hp_optimizer_7c10a_00000,RUNNING,192.168.120.84:2207,1,crossentropyloss,sgd,4,123.158,59.7087,0.08,2.23792


Result for _hp_optimizer_7c10a_00000:
  date: 2021-09-29_19-46-08
  done: true
  experiment_id: 1f6a6823b4544df88fecde2ffef3b208
  experiment_tag: 0_learner=1,loss=crossentropyloss,optimizer=sgd
  hostname: Anikets-Turing-Machine.local
  iterations_since_restore: 4
  node_ip: 192.168.120.84
  pid: 2207
  should_checkpoint: true
  time_since_restore: 123.15836071968079
  time_this_iter_s: 28.50962471961975
  time_total_s: 123.15836071968079
  timestamp: 1632924968
  timesteps_since_restore: 0
  train_loss: 2.237922847248586
  training_iteration: 4
  trial_id: 7c10a_00000
  val_accuracy: 0.08
  val_loss: 59.70871727347269
  
(pid=2207) Epoch Progress... ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 ┏━━━━━━━┳━━━━━━━┳━━━━━━┓
(pid=2207)                                                         ┃ epoch ┃ trai… ┃ val… ┃
(pid=2207)                                                         ┡━━━━━━━╇━━━━━━━╇━━━━━━┩
(pid=2207)                                                         │ 3     │  2.2… │  59… │
(

2021-09-29 19:46:09,363	WARNING util.py:163 -- The `process_trial_save` operation took 0.501 s, which may be a performance bottleneck.


Trial name,status,loc,learner,loss,optimizer,iter,total time (s),val_loss,val_accuracy,train_loss
_hp_optimizer_7c10a_00000,TERMINATED,,1,crossentropyloss,sgd,4,123.158,59.7087,0.08,2.23792


2021-09-29 19:46:09,481	INFO tune.py:561 -- Total run time: 133.22 seconds (132.27 seconds for the tuning loop).


In [ ]:
model.tuner